<a href="https://colab.research.google.com/github/guiraposo/DataScienceProjects/blob/main/Movie_Recommendation/Movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Get datafiles

!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-100k.zip
!unzip ml-1m.zip
!ls

--2024-02-23 18:37:53--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  9.79MB/s    in 0.5s    

2024-02-23 18:37:54 (9.79 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

--2024-02-23 18:37:54--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  11.7MB/s    in 0.5s    

2024-02-23 18:37:55 (11.7 MB/s) - ‘ml-1m.zip’ saved [5917

In [2]:
#Import relevant libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [3]:
#Import the datasets
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')


In [4]:
#Preparing the training set and testing set

#The training and testing sets.
#They are separated with tab. Use delimeter parameter.
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
#Convert to array to use with pytorch
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')

In [5]:
# Extract the maximum number of users and movies
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))

In [6]:
#Convert the data to an array with users in lines and movies in columns
#In this form the data willbe suitable to be inputed in a neural network

def convert(dataset):
  #This function will
    new_data = []
    for id_user in range(1, nb_users + 1):
        id_movies = dataset[:,1][dataset[:,0] == id_user]
        id_ratings = dataset[:,2][dataset[:,0] == id_user]
        rating_list = np.zeros(nb_movies)
        rating_list[id_movies - 1] = id_ratings
        new_data.append(list(rating_list))
    return new_data

In [7]:
training_set= convert(training_set)
test_set = convert(test_set)


In [13]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [15]:
# Convert to binary classification
training_set[training_set==0] = -1
training_set[training_set==1] = 0
training_set[training_set==2] = 0
training_set[training_set>=3] = 1
test_set[test_set==0] = -1
test_set[test_set==1] = 0
test_set[test_set==2] = 0
test_set[test_set>=3] = 1